In [1]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs
import jieba.posseg as pseg 


# Seperate words for the text

In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拆分句子
# 版本为python3，如果为python2需要在字符串前面加上u
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('([，！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")


# Calculate the length of the sentenct

In [4]:
def sentence_length(text):
    dict_sentence_len=dict()
    for line in text: #这个循环是遍历整个文本的，inputs是整个文本，line是文本中的每一段
        #print(line)    
        l=cut_sent(line)   # l是给一段文本进行拆分句子后的结果
        for w in l:  #这个循环是遍历一段文字里的每个句子,l是一段里的所有句子存放的list，w是每个句子
            line_seg = seg_sentence(w)  #line_seg 是一个句子拆分词语后的结果
            words=list(line_seg.split(' ')) #将一个句子中的词语拆分后逐个存放在list中
            dict_sentence_len[line_seg]=len(words)+1  #本句的长度为这个list的长度（+1是因为句子从0开始）
    return dict_sentence_len

# Transform the dict into dataframe and save the results of sentence-length into .xls file

In [5]:
# Read texts

# Save the book names in a list
file_names=['哈利波特与魔法石','哈利波特与密室','哈利波特与阿兹卡班的囚徒','哈利波特与火焰杯',
           '哈利波特与凤凰社','哈利波特与混血王子','哈利波特与死亡圣器']
for file_name in file_names:
    chapter_no=1
    output_name = './dataset/哈利波特/output/句子长度/'+file_name+' 句子长度.xlsx'
    file_path = './dataset/哈利波特/'+file_name+'(人文版).txt'
    text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
    sentence_len=sentence_length(text)
    results_df=pd.DataFrame(columns = ['句子','长度']) #为了方便存储，我把dict转成了dataframe格式，最后存放在excel表格里
    results_df['句子']=sentence_len.keys()
    results_df['长度']=sentence_len.values()
    results_df.to_excel(output_name)

# Calculate flags for the text

In [8]:
# calculate flags in each sentence
def flag_counting(each_sentence):
    # words_flags is the results of pseg.cut()
    words_flags = pseg.cut(each_sentence)
    #word presents each word, while flag means the property of the word
    flag_counting=dict()
    for word,flag in words_flags:    
        if flag not in flag_counting:
            flag_counting[flag] = 1
        else:
            flag_counting[flag] += 1
    
    return flag_counting

## Generate tables for the results of the flag_calculating

In [13]:
def table_generation(sentence_dict, flag_counting):
    df = pd.DataFrame(columns = ['句子'])
    df['句子']=sentence_dict
    line_num = 0
    for line in flag_counting:
        #k presents each flag in one sentence.
        for k in line.keys():
            
            if k not in df.columns:
                df[k] = 0
            df[k][line_num] = line[k]       
        line_num +=1
    return df

## Generate the results of flags_calculator and save it into the .txt file

In [14]:
file_name='春'
#output_name =  file_name+' 句子长度.xlsx'
file_path = './dataset/课文/'+file_name+'.txt'
text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
#df = pd.DataFrame(columns = ['句子','词性'])
#df = pd.DataFrame(columns = ['句子'])
sentence_arr = []
labels = []
sentence_list= []
pseg_list = []
line_counting = 0
for block in text:
    sentences_list =cut_sent(block)
    # sentence is each sentence
    for sentence in sentences_list:

        sentence_list.append(sentence)
        flag_result = flag_counting(sentence)

        pseg_list.append(flag_result)
#df['句子'] = sentence_list
flags_result = table_generation(sentence_list, pseg_list)

output_name = './dataset/课文/' + file_name + '_flags.txt'
file = open(output_name, 'w')
for column in flags_result.columns:
    #print(str(list(flags_result[column])))
    file.write(str(list(flags_result[column])))   
file.close()

n
x
n
x
n
v
ul
x
t
uj


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


n
a
ul
x
r
d
v
uj
n
x
d
nr
ul
n
x
nr
v
ul
x
n
v
ul
x
ns
uj
n
v
ul
x
n
i
p
v
x
z
x
z
x
n
f
x
nr
f
x
v
x
m
n
a
v
uj
x
v
uz
x
v
uz
x
v
m
x
v
n
x
nz
m
v
x
v
m
x
n
z
x
n
z
uj
x
n
x
r
v
x
r
v
x
d
v
ul
n
x
a
uj
v
n
x
n
uj
v
x
a
uj
v
n
x
nr
v
uz
n
x
ul
s
d
a
v
n
x
v
f
i
uj
n
z
uz
x
b
uj
n
i
x
n
v
x
v
n
uj
x
v
n
uj
x
v
p
l
x
v
n
x
v
nz
x
d
v
y
uj
x
x
i
nr
a
uj
x
v
n
uj
uz
r
x
n
f
v
q
a
uj
x
v
uz
n
x
v
r
uj
n
x
d
p
nz
uj
n
f
v
x
n
d
nr
p
s
x
b
v
ul
x
nr
n
v
a
uj
x
v
uj
n
x
p
n
v
uz
x
i
n
uj
x
r
d
t
a
uv
v
uz
x
n
v
d
a
uj
x
m
d
x
v
x
v
x
v
nr
x
v
n
x
v
n
x
ad
uv
v
uz
x
n
f
a
v
uz
m
x
n
d
a
ud
v
x
n
d
a
ud
v
r
uj
x
t
x
v
ul
x
m
n
uj
x
v
m
nr
c
nz
uj
n
x
p
s
x
a
s
x
ns
d
x
v
n
d
uz
uj
x
s
v
vn
uj
n
x
v
n
x
r
uj
n
x
z
uj
p
n
f
nr
uz
x
s
n
d
m
ul
x
s
n
d
m
ul
x
s
x
n
x
a
b
x
d
v
u
x
m
d
v
ul
x
a
l
x
v
i
x
r
v
uj
m
t
x
x
i
d
v
n
x
l
n
x
l
v
x
t
v
d
n
uj
x
l
d
v
a
uj
x
r
n
uz
x
t
v
nr
x
i
uj
x
v
uz
x
v
uz
x
t
v
a
uj
x
v
n
a
uj
c
x
v
uz
r
f
t
x


In [12]:
flags_result.columns

Index(['句子', 'n', 'x', 'v', 'ul', 't', 'uj', 'a', 'r', 'd', 'nr', 'ns', 'i',
       'p', 'z', 'f', 'm', 'uz', 'nz', 's', 'b', 'l', 'y', 'q', 'uv', 'ad',
       'ud', 'c', 'vn', 'u'],
      dtype='object')